## Ankara için bir kira tahmini uygulaması

In [1]:
import os
import numpy as np
import pandas as pd

In [67]:
#csv dosyasındaki verileri 
df = pd.read_csv('hepsiEmlakAnkaraKiralikDaireler.csv')

In [68]:
#sütunları inceleme
print(df.columns)               

Index(['img-link href', 'photo-count', 'list-view-price', 'list-view-date',
       'list-view-title', 'left', 'celly', 'celly 2', 'celly 3', 'celly 4',
       'list-view-location', 'img-wrp href', 'he-lazy-image src', 'wp-btn',
       'listing-card--owner-info__firm-name', 'img-wrp href 2',
       'he-lazy-image src 2', 'listing-card--owner-info__name',
       'he-lazy-image src 3', 'list-view-header'],
      dtype='object')


In [70]:
#gereksiz sütunları kaldırma
df.drop(['photo-count', 'list-view-date', 'left', 'list-view-location', 'img-wrp href', 'he-lazy-image src', 'wp-btn', 'listing-card--owner-info__firm-name', 'img-wrp href 2', 'he-lazy-image src 2', 'listing-card--owner-info__name', 'he-lazy-image src 3', 'list-view-header','list-view-title'], axis=1,inplace=True)

In [69]:
#adresleri yeni bi df yapalım
addressDf = pd.DataFrame(df['list-view-location'].str.split('/', expand=True))
addressDf.columns = ["İl","İlçe","Mahalle"]
#burada tüm il sütununda Ankara yazdığı için belirli bir ölçüt değildir, zaten programın amacı Ankara'daki iller olduğu
#için il sütununu siliyoruz
addressDf.drop(columns="İl", inplace=True)
addressDf


,İlçe,Mahalle
0,Çankaya,Fakülteler Mah.
1,Çankaya,Bahçelievler Mah.
2,Keçiören,Yeşiltepe Mah.
3,Mamak,Ege Mah.
4,Çankaya,Gaziosmanpaşa Mah.
...,...,...
10325,Çankaya,Mutlukent Mah.
10326,Altındağ,Aydınlıkevler Mah.
10327,Keçiören,Kavacık Subayevleri Mah.
10328,Yenimahalle,Emniyet Mah.


In [71]:
#adresleri ve orijinal dataframe'imizi birleştirelim
df = pd.concat([df,addressDf], axis = 1)
df.head()

,img-link href,list-view-price,celly,celly 2,celly 3,celly 4,İlçe,Mahalle
0,https://www.hepsiemlak.com/ankara-cankaya-faku...,25.000,3 +\n 1,120 m²,40\n Yaşında,3. Kat,Çankaya,Fakülteler Mah.
1,https://www.hepsiemlak.com/ankara-cankaya-bahc...,38.000,2 +\n 1,90 m²,30\n Yaşında,1. Kat,Çankaya,Bahçelievler Mah.
2,https://www.hepsiemlak.com/ankara-kecioren-yes...,13.750,3 +\n 1,130 m²,25\n Yaşında,Yüksek Giriş,Keçiören,Yeşiltepe Mah.
3,https://www.hepsiemlak.com/ankara-mamak-ege-ki...,26.500,3 +\n 1,130 m²,13\n Yaşında,5. Kat,Mamak,Ege Mah.
4,https://www.hepsiemlak.com/ankara-cankaya-gazi...,55.000,5 +\n 1,225 m²,35\n Yaşında,Bahçe Katı,Çankaya,Gaziosmanpaşa Mah.


In [72]:
df.rename(columns={
    "img-link href": "Link",
    "list-view-price": "Fiyat",
    "celly": "Oda Sayısı",
    "celly 2": "m^2",
    "celly 3": "Bina Yaşı",
    "celly 4": "Kat"
}, inplace=True)

In [75]:
print(df['Oda Sayısı'].unique())

['3 +\n                          1' '2 +\n                          1'
 '5 +\n                          1' '4 +\n                          1'
 '1 +\n                          1' '4 +\n                          4'
 '6 +\n                          1' 'Stüdyo'
 '3 +\n                          2' '9 +\n                          3'
 '5 +\n                          0' '5 +\n                          2'
 '4 +\n                          0' '4 +\n                          2'
 '6 +\n                          2' '8 +\n                          1'
 '3 +\n                          0' '7 +\n                          1'
 '3 +\n                          3' '2 +\n                          0'
 '11 +\n                          3' '3 +\n                          5'
 '4 +\n                          5' '7 +\n                          2'
 '6 +\n                          6' '2 +\n                          2'
 '2 +\n                          3' '10 +\n                          1'
 '1 +\n                       

In [77]:
df['Oda Sayısı'] = df['Oda Sayısı'].apply(lambda x: x.replace('Stüdyo', '1 + 0') )
df['Oda Sayısı'] = df['Oda Sayısı'].apply(lambda x: x.replace('\n', '') )

In [88]:
df['Oda'] = df['Oda Sayısı'].apply(lambda x: x.split('+')[0] ).astype(int)
df['Salon'] = df['Oda Sayısı'].apply(lambda x: x.split('+')[1] ).astype(int)

In [ ]:
df.drop(columns='Oda Sayısı', inplace=True)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10330 entries, 0 to 10329
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Link       10330 non-null  object
 1   Fiyat      10330 non-null  object
 2   m^2        10330 non-null  object
 3   Bina Yaşı  10326 non-null  object
 4   Kat        10127 non-null  object
 5   İlçe       10330 non-null  object
 6   Mahalle    10330 non-null  object
 7   Oda        10330 non-null  int64 
 8   Salon      10330 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 726.5+ KB


In [98]:
df['m^2'].unique()

array(['120 m²', '90 m²', '130 m²', '225 m²', '110 m²', '220 m²',
       '200 m²', '105 m²', '160 m²', '150 m²', '135 m²', '145 m²',
       '65 m²', '85 m²', '118 m²', '75 m²', '125 m²', '40 m²', '95 m²',
       '155 m²', '115 m²', '140 m²', '45 m²', '180 m²', '175 m²',
       '100 m²', '350 m²', '43 m²', '168 m²', '210 m²', '165 m²',
       '190 m²', '1.450 m²', '170 m²', '195 m²', '138 m²', '285 m²',
       '330 m²', '640 m²', '70 m²', '55 m²', '108 m²', '3.540 m²',
       '60 m²', '700 m²', '35 m²', '162 m²', '62 m²', '260 m²', '300 m²',
       '83 m²', '230 m²', '80 m²', '128 m²', '450 m²', '127 m²', '50 m²',
       '139 m²', '54 m²', '185 m²', '240 m²', '98 m²', '107 m²', '313 m²',
       '151 m²', '412 m²', '250 m²', '164 m²', '30 m²', '96 m²', '154 m²',
       '159 m²', '82 m²', '183 m²', '58 m²', '72 m²', '67 m²', '117 m²',
       '158 m²', '1.105 m²', '86 m²', '276 m²', '360 m²', '149 m²',
       '94 m²', '124 m²', '280 m²', '99 m²', '1.120 m²', '126 m²',
       '122 m²', '375

In [104]:
df['m^2'] = df['m^2'].apply(lambda x: x.replace('m²', ''))
df['m^2'] = df['m^2'].apply(lambda x: x.replace('.', '')).astype(int)


In [105]:
df['m^2'].unique()

array([ 120,   90,  130,  225,  110,  220,  200,  105,  160,  150,  135,
        145,   65,   85,  118,   75,  125,   40,   95,  155,  115,  140,
         45,  180,  175,  100,  350,   43,  168,  210,  165,  190, 1450,
        170,  195,  138,  285,  330,  640,   70,   55,  108, 3540,   60,
        700,   35,  162,   62,  260,  300,   83,  230,   80,  128,  450,
        127,   50,  139,   54,  185,  240,   98,  107,  313,  151,  412,
        250,  164,   30,   96,  154,  159,   82,  183,   58,   72,   67,
        117,  158, 1105,   86,  276,  360,  149,   94,  124,  280,   99,
       1120,  126,  122,  375,  112,  132,  292,  102,  750,  123,  650,
        500,  430,  900,   97,  270,  167,  103,  113,   92,   89,   93,
         57,  161,  290,  232,  144,   78,  114,   66,  104, 1200,  142,
        137,  470,  116,  141,  255,  146,  235,  166,  400,   79,  310,
        242,   26,   52,  133,  197,   56,  216,  119,  101,  169,  245,
        163,  380,  184,  157,  399,  129,  201,  1

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10330 entries, 0 to 10329
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Link       10330 non-null  object
 1   Fiyat      10330 non-null  object
 2   m^2        10330 non-null  int64 
 3   Bina Yaşı  10326 non-null  object
 4   Kat        10127 non-null  object
 5   İlçe       10330 non-null  object
 6   Mahalle    10330 non-null  object
 7   Oda        10330 non-null  int64 
 8   Salon      10330 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 726.5+ KB


In [108]:
df

,Link,Fiyat,m^2,Bina Yaşı,Kat,İlçe,Mahalle,Oda,Salon
0,https://www.hepsiemlak.com/ankara-cankaya-faku...,25.000,120,40\n Yaşında,3. Kat,Çankaya,Fakülteler Mah.,3,1
1,https://www.hepsiemlak.com/ankara-cankaya-bahc...,38.000,90,30\n Yaşında,1. Kat,Çankaya,Bahçelievler Mah.,2,1
2,https://www.hepsiemlak.com/ankara-kecioren-yes...,13.750,130,25\n Yaşında,Yüksek Giriş,Keçiören,Yeşiltepe Mah.,3,1
3,https://www.hepsiemlak.com/ankara-mamak-ege-ki...,26.500,130,13\n Yaşında,5. Kat,Mamak,Ege Mah.,3,1
4,https://www.hepsiemlak.com/ankara-cankaya-gazi...,55.000,225,35\n Yaşında,Bahçe Katı,Çankaya,Gaziosmanpaşa Mah.,5,1
...,...,...,...,...,...,...,...,...,...
10325,https://www.hepsiemlak.com/ankara-cankaya-mutl...,155.000,313,6\n Yaşında,5. Kat,Çankaya,Mutlukent Mah.,4,2
10326,https://www.hepsiemlak.com/ankara-altindag-ayd...,17.500,100,24\n Yaşında,Kot 1,Altındağ,Aydınlıkevler Mah.,3,1
10327,https://www.hepsiemlak.com/ankara-kecioren-kav...,17.500,100,24\n Yaşında,Kot 1,Keçiören,Kavacık Subayevleri Mah.,3,1
10328,https://www.hepsiemlak.com/ankara-yenimahalle-...,60.000,161,Sıfır Bina,5. Kat,Yenimahalle,Emniyet Mah.,2,1
